In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive') 

In [ ]:
# !kill process_id

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [ ]:
! pip install datasets transformers

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then uncomment the following cell and input your username and password (this only works on Colab, in a regular notebook, you need to do this in a terminal):

In [ ]:
# !huggingface-cli login

Then you need to install Git-LFS and setup Git if you haven't already. Uncomment the following instructions and adapt with your name and email:

In [ ]:
!apt install git-lfs
!git config --global user.email "jirarotej@gmail.com"
!git config --global user.name "Jirarote Jirasirikul"

Make sure your version of Transformers is at least 4.8.1 since the functionality was introduced in that version:

In [ ]:
import transformers

print(transformers.__version__)

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/text-classification).

# Fine-tuning a model on a text classification task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a text classification task of the [GLUE Benchmark](https://gluebenchmark.com/).

![Widget inference on a text classification task](https://github.com/huggingface/notebooks/blob/master/examples/images/text_classification.png?raw=1)

The GLUE Benchmark is a group of nine classification tasks on sentences or pairs of sentences which are:

- [CoLA](https://nyu-mll.github.io/CoLA/) (Corpus of Linguistic Acceptability) Determine if a sentence is grammatically correct or not.is a  dataset containing sentences labeled grammatically correct or not.
- [MNLI](https://arxiv.org/abs/1704.05426) (Multi-Genre Natural Language Inference) Determine if a sentence entails, contradicts or is unrelated to a given hypothesis. (This dataset has two versions, one with the validation and test set coming from the same distribution, another called mismatched where the validation and test use out-of-domain data.)
- [MRPC](https://www.microsoft.com/en-us/download/details.aspx?id=52398) (Microsoft Research Paraphrase Corpus) Determine if two sentences are paraphrases from one another or not.
- [QNLI](https://rajpurkar.github.io/SQuAD-explorer/) (Question-answering Natural Language Inference) Determine if the answer to a question is in the second sentence or not. (This dataset is built from the SQuAD dataset.)
- [QQP](https://data.quora.com/First-Quora-Dataset-Release-Question-Pairs) (Quora Question Pairs2) Determine if two questions are semantically equivalent or not.
- [RTE](https://aclweb.org/aclwiki/Recognizing_Textual_Entailment) (Recognizing Textual Entailment) Determine if a sentence entails a given hypothesis or not.
- [SST-2](https://nlp.stanford.edu/sentiment/index.html) (Stanford Sentiment Treebank) Determine if the sentence has a positive or negative sentiment.
- [STS-B](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark) (Semantic Textual Similarity Benchmark) Determine the similarity of two sentences with a score from 1 to 5.
- [WNLI](https://cs.nyu.edu/faculty/davise/papers/WinogradSchemas/WS.html) (Winograd Natural Language Inference) Determine if a sentence with an anonymous pronoun and a sentence with this pronoun replaced are entailed or not. (This dataset is built from the Winograd Schema Challenge dataset.)

We will see how to easily load the dataset for each one of those tasks and use the `Trainer` API to fine-tune a model on it. Each task is named by its acronym, with `mnli-mm` standing for the mismatched version of MNLI (so same training set as `mnli` but different validation and test sets):

In [ ]:
# GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

This notebook is built to run on any of the tasks in the list above, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a classification head. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [ ]:
import os
import pandas as pd
from datasets import load_dataset, load_metric, Dataset, DatasetDict, ClassLabel

Apart from `mnli-mm` being a special code, we can directly pass our task name to those functions. `load_dataset` will cache the dataset to avoid downloading it again the next time you run this cell.

In [ ]:
# actual_task = "mnli" if task == "mnli-mm" else task
# dataset = load_dataset("glue", actual_task)
# metric = load_metric('glue', actual_task)

### pubmedqa

In [ ]:
DATAPATH = "/content/drive/MyDrive/MinorThesis/"
DATASET = "pubmedqa"

# Train & Valid
list_data_fold = []
for i in range(10): # We merge dataset to generate trained (Separate later using index - filename_line)
    temppath_train = os.path.join(DATAPATH,"datasets","raw",DATASET,"pqal_fold"+str(i),"train_set.json")
    temppath_valid = os.path.join(DATAPATH,"datasets","raw",DATASET,"pqal_fold"+str(i),"dev_set.json")
        
    df_temp_train = pd.read_json(temppath_train).transpose()
    df_temp_valid = pd.read_json(temppath_valid).transpose()
    list_data_fold.append((df_temp_train,df_temp_valid))
    # print(df_temp_train.shape,df_temp_valid.shape)

# Test
temppath_test = os.path.join(DATAPATH,"datasets","raw",DATASET,'test_set.json')
df_test = pd.read_json(temppath_test).transpose()

In [ ]:
def get_pubmedqa_fold(list_data_fold,data_test,fold_no = 0):
    # temppath_test = os.path.join(DATAPATH,"datasets","raw",DATASET,"test_set.json")
    df_train = list_data_fold[fold_no][0]
    df_valid = list_data_fold[fold_no][1]
    df_test = data_test

    df_train['CONTEXTS_JOINED'] = df_train.CONTEXTS.apply(lambda x : (' ').join(x))
    df_valid['CONTEXTS_JOINED'] = df_valid.CONTEXTS.apply(lambda x : (' ').join(x))
    df_test['CONTEXTS_JOINED'] = df_test.CONTEXTS.apply(lambda x : (' ').join(x))
    
    # # REASONING REQUIRED
    # df_train_mod = df_train[['QUESTION','CONTEXTS','final_decision','reasoning_required_pred','reasoning_free_pred']].reset_index().copy()
    # df_train_mod['text'] = df_train_mod.QUESTION +". "+ df_train_mod.CONTEXTS.apply(lambda x : (' ').join(x)) #question before
    # # df_train_mod['text'] = df_train_mod.CONTEXTS.apply(lambda x : (' ').join(x)) +" "+ df_train_mod.QUESTION  #question after
    # df_train_mod.drop(columns=['QUESTION','CONTEXTS'],inplace=True)
    # df_train_mod.columns = ['id','label','reasoning_required_pred','reasoning_free_pred','text']

    # df_valid_mod = df_valid[['QUESTION','CONTEXTS','final_decision','reasoning_required_pred','reasoning_free_pred']].reset_index().copy()
    # df_valid_mod['text'] = df_valid_mod.QUESTION +". "+ df_valid_mod.CONTEXTS.apply(lambda x : (' ').join(x)) #question before
    # # df_train_mod['text'] = df_valid_mod.CONTEXTS.apply(lambda x : (' ').join(x)) +" "+ df_valid_mod.QUESTION  #question after
    # df_valid_mod.drop(columns=['QUESTION','CONTEXTS'],inplace=True)
    # df_valid_mod.columns = ['id','label','reasoning_required_pred','reasoning_free_pred','text']

    # df_test_mod = df_test[['QUESTION','CONTEXTS','final_decision','reasoning_required_pred','reasoning_free_pred']].reset_index().copy()
    # df_test_mod['text'] = df_test_mod.QUESTION +". "+ df_test_mod.CONTEXTS.apply(lambda x : (' ').join(x)) #question before
    # # df_test_mod['text'] = df_test_mod.CONTEXTS.apply(lambda x : (' ').join(x)) +" "+ df_test_mod.QUESTION  #question after
    # df_test_mod.drop(columns=['QUESTION','CONTEXTS'],inplace=True)
    # df_test_mod.columns = ['id','label','reasoning_required_pred','reasoning_free_pred','text']

    # id2label={ 0: "no", 1: "maybe", 2: "yes"}
    # label2id = dict((v,k) for k,v in id2label.items())
    # df_train_mod['label'] = df_train_mod.label.apply(lambda x : label2id[x])
    # df_valid_mod['label'] = df_valid_mod.label.apply(lambda x : label2id[x])
    # df_test_mod['label'] = df_test_mod.label.apply(lambda x : label2id[x])

    dataset = DatasetDict({
        'train':Dataset.from_pandas(df_train),
        'validation':Dataset.from_pandas(df_valid),
        'test':Dataset.from_pandas(df_test),
    })

    dataset.remove_columns_(['CONTEXTS','LABELS', 'MESHES','YEAR', 'reasoning_required_pred', 'reasoning_free_pred', 'LONG_ANSWER','__index_level_0__'])
    dataset.rename_column_("final_decision", "labels")
    return dataset.class_encode_column("labels")

In [ ]:
dataset = get_pubmedqa_fold(list_data_fold,df_test,0)
dataset.num_rows

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

To access an actual element, you need to select a split first, then give an index:

In [ ]:
dataset["train"].features

In [ ]:
dataset["train"][0]

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
# show_random_elements(dataset["train"])

The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [ ]:
# https://github.com/huggingface/datasets/tree/master/metrics
from datasets import load_metric
metric = load_metric("accuracy")

# metric = load_metric("https://github.com/huggingface/datasets/blob/master/metrics/accuracy/accuracy.py")
# Example of typical usage
# for batch in dataset:
#     inputs, references = batch
#     predictions = model(inputs)
#     metric.add_batch(predictions=predictions, references=references)
# score = metric.compute()

In [ ]:
metric

You can call its `compute` method with your predictions and labels directly and it will return a dictionary with the metric(s) value:

In [ ]:
import numpy as np

fake_preds = np.random.randint(0, 3, size=(64,))
fake_labels = np.random.randint(0, 3, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

## Fine-tuning the model

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # predictions = predictions[:, 0]
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

#### For all folds

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
task = "pubmedqa"
model_checkpoint = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
# model_checkpoint = "dmis-lab/biobert-base-cased-v1.1"
# model_checkpoint = "dmis-lab/biobert-v1.1"

LEARNING_RATE = 3e-3
BATCH_SIZE = 8
NUM_LABELS = 3
MAX_LENGTH = 512

metric_name = "accuracy"
model_name = model_checkpoint.split("/")[-1]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

def preprocess_function(examples):
    return tokenizer(examples["QUESTION"], examples["CONTEXTS_JOINED"], truncation=True,padding=True,max_length=MAX_LENGTH)

In [ ]:
list_acc = []
for i in range(10):
    torch.cuda.empty_cache()
    print("###### FOLD",i,"######")
    dataset = get_pubmedqa_fold(list_data_fold,df_test,i)
    print("num_rows",dataset.num_rows)

    encoded_dataset = dataset.map(preprocess_function, batched=True)
    encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=NUM_LABELS)

    args = TrainingArguments(
        f"{model_name}-finetuned-{task}",
        evaluation_strategy = "epoch",
        save_strategy = "epoch",
        learning_rate=LEARNING_RATE,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        eval_accumulation_steps=1,
        num_train_epochs=10,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model=metric_name,
        # The next line is important to ensure the dataset labels are properly passed to the model
        remove_unused_columns=True,
        # push_to_hub=True,
        # push_to_hub_model_id=f"{model_name}-finetuned-{task}-2",
    )

    trainer = Trainer(
        model,
        args,
        train_dataset=encoded_dataset["train"],
        eval_dataset=encoded_dataset["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    commit_msg = ""
    commit_msg += "BATCH_SIZE="+str(BATCH_SIZE)+"\n"
    commit_msg += "LEARNING_RATE="+str(LEARNING_RATE)+"\n"
    commit_msg += "MAX_LENGTH="+str(MAX_LENGTH)+"\n"
    commit_msg += "FOLD="+str(i)+"\n"

    # trainer.push_to_hub(commit_msg)
    output = trainer.evaluate(encoded_dataset['test'])
    list_acc.append(output['eval_accuracy'])


In [ ]:
list_acc

In [ ]:
np.mean(list_acc)